In [ ]:
from datasets import load_dataset
from transformers import FSMTForConditionalGeneration, FSMTTokenizer
from lm_polygraph.estimators import *
from lm_polygraph.utils.model import WhiteboxModel
from lm_polygraph.utils.dataset import Dataset
from lm_polygraph.utils.processor import Logger
from lm_polygraph.utils.manager import UEManager
from lm_polygraph.ue_metrics import PredictionRejectionArea
from lm_polygraph.generation_metrics import RougeMetric, BartScoreSeqMetric, ModelScoreSeqMetric, ModelScoreTokenwiseMetric

# Specify HyperParameters

In [ ]:
model_path = "facebook/wmt19-en-de"
device = "cpu"
dataset_name = ("wmt14", "de-en")
batch_size = 4
seed = 42

# Initialize Model

In [ ]:
base_model = FSMTForConditionalGeneration.from_pretrained(
    model_path,
    device_map=device,
)
tokenizer = FSMTTokenizer.from_pretrained(model_path)

model = WhiteboxModel(base_model, tokenizer, model_type="Seq2SeqLM")

# Train and Eval Datasets

In [ ]:
hf_dataset = load_dataset(*dataset_name)

dataset = Dataset(
    x=[txt["en"] for txt in hf_dataset["test"]["translation"]],
    y=[txt["de"] for txt in hf_dataset["test"]["translation"]],
    batch_size=batch_size,
)
dataset.subsample(16, seed=seed)

train_dataset = Dataset(
    x=[txt["en"] for txt in hf_dataset["train"]["translation"][:1000]],
    y=[txt["de"] for txt in hf_dataset["train"]["translation"][:1000]],
    batch_size=batch_size,
)
train_dataset.subsample(16, seed=seed)

# Metric, UE Metric, and UE Methods

In [ ]:
ue_methods = [MaximumSequenceProbability(), 
              SemanticEntropy(),
              MahalanobisDistanceSeq("encoder"),]

ue_metrics = [PredictionRejectionArea()]

metrics = [RougeMetric('rougeL')]

loggers = [Logger()] 

# Initialize UE Manager

In [ ]:
man = UEManager(
    dataset,
    model,
    ue_methods,
    metrics,
    ue_metrics,
    loggers,
    train_data=train_dataset,
)

# Compute Results

In [ ]:
results = man()

In [ ]:
for key in results.keys():
    print(f"UE Score: {key[1]}, Metric: {key[2]}, UE Metric: {key[3]}, Score: {results[key]:.3f}")